# Simulating follow-on interviews with survey respondents
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for creating AI agents representing survey respondents and simulating follow-on interviews with them.

[EDSL is an open-source libary](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Importing survey data
We start by using the `Conjure` module to import a dataset of survey responses and convert it into EDSL objects. 
For purposes of demonstration we'll use a set of mock responses to a reader survey about a home renovation newsletter:

In [1]:
# "How often do you read our home renovation tips newsletter?","On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)","Would you recommend our newsletter to a friend or family member?","What topics would you like to see covered in future issues of our newsletter?"
# "Weekly",9,"Yes","More articles on upcycling old furniture, please! I'm all about that eco-friendly life."
# "Monthly",7,"Yes","How about some tips for renovating tiny apartments? Not all of us live in mansions, you know!"
# "Quarterly",4,"No","Honestly, I find most of the content too advanced. Could you include a 'Renovation 101' section for us newbies?"
# "Rarely",2,"No","Your newsletter is way too text-heavy. Ever heard of infographics? Or videos? Join the 21st century, folks!"
# "Weekly",10,"Yes","I'd kill for a deep dive into historical home restoration techniques. Let's get nerdy with it!"
# "Monthly",6,"No","Meh, it's okay. Maybe feature some celebrity home renos? That'd spice things up a bit."
# "Quarterly",8,"Yes","As a contractor, I'd love to see a section addressing common client misconceptions about renovations."
# "Weekly",9,"Yes","Your content rocks, but how about some love for us renters? Removable upgrades are my jam!"
# "Monthly",5,"No","Too many ads, not enough substance. I feel like I'm reading a catalog half the time."
# "Rarely",3,"No","Snooze fest. Where's the excitement? How about some extreme home makeover challenges?"
# "Weekly",10,"Yes","I'm obsessed with smart home tech. Can we get more articles on integrating tech into older homes?"
# "Monthly",7,"Yes","Love the newsletter, but please proofread! The typos are driving my inner grammar nerd crazy."
# "Quarterly",6,"Yes","How about a series on renovation horror stories and how to avoid them? Could be both helpful and entertaining!"
# "Rarely",1,"No","Unsubscribe button, where art thou? This newsletter is about as useful as a chocolate teapot."
# "Monthly",8,"Yes","Any chance of including a Q&A section with real architects or designers? I've got questions!"
# "Weekly",9,"Yes","More budget breakdowns, please! I want to know exactly where my renovation dollars are going."
# "Quarterly",5,"No","It's fine, I guess. But could you cover more international design trends? It's a big world out there!"
# "Monthly",7,"Yes","Hey, how about some tips for pet-friendly renovations? My cats are destroying everything!"
# "Weekly",10,"Yes","Absolutely adore your content! But I'm dying to see more about incorporating art into home design. Let's get creative!"
# "Rarely",4,"No","Too mainstream for my taste. Where are the articles on avant-garde design or extreme home modifications?"

In [2]:
from edsl import Conjure

In [3]:
c = Conjure("newsletter_survey_responses.csv")

We can inspect the questions that were in the survey:

In [4]:
c.question_texts

['How often do you read our home renovation tips newsletter?',
 'On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)',
 'Would you recommend our newsletter to a friend or family member?',
 'What topics would you like to see covered in future issues of our newsletter?']

## Generating EDSL objects
The `Conjure` module provides methods for automatically generating `Survey`, `Results` and `Agent` objects from the survey data. These methods allow us to use [built-in methods for analyzing the results](https://docs.expectedparrot.com/en/latest/results.html) and [create AI agents](https://docs.expectedparrot.com/en/latest/agents.html) representing the original survey respondents.

### Recreating results
For example, we can recreate the responses as EDSL `Results` and sample, filter, select and print them:

In [5]:
results = c.to_results()

In [6]:
(results
 .sample(3)
 .select("answer.*")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                    ┃ answer                     ┃ answer                    ┃ answer                     ┃
┃ .topics_would_like_see_c… ┃ .scale_1_10_useful_find_c… ┃ .often_read_home_renovat… ┃ .would_recommend_newslett… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Absolutely adore your     │ 10                         │ Weekly                    │ Yes                        │
│ content! But I'm dying to │                            │                           │                            │
│ see more about            │                            │                           │                            │
│ incorporating art into    │                            │                           │                            │
│ home design. Let's get    │                            │                           │                            │
│ creative!                 │                            │                           │                            │
├───────────────────────────┼────────────────────────────┼───────────────────────────┼────────────────────────────┤
│ I'm obsessed with smart   │ 10                         │ Weekly                    │ Yes                        │
│ home tech. Can we get     │                            │                           │                            │
│ more articles on          │                            │                           │                            │
│ integrating tech into     │                            │                           │                            │
│ older homes?              │                            │                           │                            │
├───────────────────────────┼────────────────────────────┼───────────────────────────┼────────────────────────────┤
│ Snooze fest. Where's the  │ 3                          │ Rarely                    │ No                         │
│ excitement? How about     │                            │                           │                            │
│ some extreme home         │                            │                           │                            │
│ makeover challenges?      │                            │                           │                            │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

### Designing AI agents
We can also generate an `AgentList` for the respondents, each containing a dictionary of `traits` for a respondent's survey answers and a `codebook` for the questions that were asked:

In [7]:
agents = c.to_agent_list()

We can inspect some of them:

In [8]:
agents[0:2]

AgentList([Agent(traits = {'often_read_home_renovation_tips': 'Weekly', 'scale_1_10_useful_find_content': 9, 'would_recommend_newsletter_friend': 'Yes', 'topics_would_like_see_covered': "More articles on upcycling old furniture, please! I'm all about that eco-friendly life."}, codebook = {'often_read_home_renovation_tips': 'How often do you read our home renovation tips newsletter?', 'scale_1_10_useful_find_content': 'On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)', 'would_recommend_newsletter_friend': 'Would you recommend our newsletter to a friend or family member?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues of our newsletter?'}), Agent(traits = {'often_read_home_renovation_tips': 'Monthly', 'scale_1_10_useful_find_content': 7, 'would_recommend_newsletter_friend': 'Yes', 'topics_would_like_see_covered': 'How about some tips for renovating tiny apartments? Not all of us live in mansions, you know!'}, codebook = {'often_read_home_renovation_tips': 'How often do you read our home renovation tips newsletter?', 'scale_1_10_useful_find_content': 'On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)', 'would_recommend_newsletter_friend': 'Would you recommend our newsletter to a friend or family member?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues of our newsletter?'})])

We can give the agents names for reference in conducting follow-on interviews:

In [9]:
for i, agent in enumerate(agents):
    setattr(agent, 'name', f"Respondent {i+1}")

In [10]:
agents[0:2]

AgentList([Agent(name = """Respondent 1""", traits = {'often_read_home_renovation_tips': 'Weekly', 'scale_1_10_useful_find_content': 9, 'would_recommend_newsletter_friend': 'Yes', 'topics_would_like_see_covered': "More articles on upcycling old furniture, please! I'm all about that eco-friendly life."}, codebook = {'often_read_home_renovation_tips': 'How often do you read our home renovation tips newsletter?', 'scale_1_10_useful_find_content': 'On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)', 'would_recommend_newsletter_friend': 'Would you recommend our newsletter to a friend or family member?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues of our newsletter?'}), Agent(name = """Respondent 2""", traits = {'often_read_home_renovation_tips': 'Monthly', 'scale_1_10_useful_find_content': 7, 'would_recommend_newsletter_friend': 'Yes', 'topics_would_like_see_covered': 'How about some tips for renovating tiny apartments? Not all of us live in mansions, you know!'}, codebook = {'often_read_home_renovation_tips': 'How often do you read our home renovation tips newsletter?', 'scale_1_10_useful_find_content': 'On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)', 'would_recommend_newsletter_friend': 'Would you recommend our newsletter to a friend or family member?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues of our newsletter?'})])

## Conducting follow-on interviews
We can use the `Conversation` module to automate an interview with each respondent agent. 
To start, we can create an agent interviewer and pair it with each respondent agent:

In [11]:
from edsl import AgentList, Agent
from edsl.conversation.Conversation import Conversation, ConversationList

In [12]:
def interview_pair(i, agents):
    respondent = agents[i]
    interviewer = Agent(
        name = "Interviewer",
        traits = {
            "motivation": f"""
            You are conducting a follow-on interview with someone who has just 
            answered a survey about a home construction newsletter. 
            You want to get more details about their feedback on the newsletter.
            Here are the survey questions and the respondent's answers:
            { respondent.codebook }
            { respondent.traits }
            """
        }
    )
        
    return AgentList([interviewer, respondent])

Next we specify the agents and number of turns for each conversation that we want to run:

In [13]:
cl = ConversationList(
    [Conversation(agent_list = interview_pair(i, agents), max_turns = 6)
    for i in range(len(agents))]
)

Now we can run the conversations and convert them into `Results` objects for inspection and analysis:

In [14]:
cl.run()

results = cl.to_results()

To see all the components of the results that have been generated:

In [15]:
results.columns

['agent.agent_instruction',
 'agent.motivation',
 'agent.often_read_home_renovation_tips',
 'agent.scale_1_10_useful_find_content',
 'agent.topics_would_like_see_covered',
 'agent.would_recommend_newsletter_friend',
 'answer.dialogue',
 'comment.k_comment',
 'generated_tokens.dialogue_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.dialogue_system_prompt',
 'prompt.dialogue_user_prompt',
 'question_options.dialogue_question_options',
 'question_text.dialogue_question_text',
 'question_type.dialogue_question_type',
 'raw_model_response.dialogue_cost',
 'raw_model_response.dialogue_one_usd_buys',
 'raw_model_response.dialogue_raw_model_response',
 'scenario.agent_name',
 'scenario.conversation',
 'scenario.conversation_index',
 'scenario.index']

As we did above, we can filter, sort, select and print the results. 
Please see documentation for [examples of other analytical methods](https://docs.expectedparrot.com/en/latest/results.html).

In [16]:
(results
 .sort_by("conversation_index", "index")
 .select("conversation_index", "index", "agent_name", "dialogue")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario            ┃ scenario ┃ scenario      ┃ answer                                                         ┃
┃ .conversation_index ┃ .index   ┃ .agent_name   ┃ .dialogue                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0                   │ 0        │ Interviewer   │ Hi there! Thank you for taking the time to chat with me today. │
│                     │          │               │ I see you read our home renovation tips newsletter weekly and  │
│                     │          │               │ rated it quite highly. That's fantastic to hear! I'd love to   │
│                     │          │               │ dive a bit deeper into your feedback. Can you tell me more     │
│                     │          │               │ about what you find most useful in our newsletter?             │
├─────────────────────┼──────────┼───────────────┼────────────────────────────────────────────────────────────────┤
│ 0                   │ 1        │ Respondent 1  │ Hi! Thanks for reaching out. I find the practical,             │
│                     │          │               │ step-by-step guides in your newsletter incredibly useful. They │
│                     │          │               │ make even the most daunting renovation projects feel           │
│                     │          │               │ manageable. I also appreciate the variety of topics you cover, │
│                     │          │               │ from big renovations to small DIY projects. The tips on        │
│                     │          │               │ budget-friendly upgrades are particularly helpful, and I love  │
│                     │          │               │ how you incorporate eco-friendly ideas.                        │
├─────────────────────┼──────────┼───────────────┼────────────────────────────────────────────────────────────────┤
│ 0                   │ 2        │ Interviewer   │ That's wonderful feedback, thank you! It's great to hear that  │
│                     │          │               │ you find the step-by-step guides and variety of topics so      │
│                     │          │               │ beneficial. You mentioned that you especially appreciate the   │
│                     │          │               │ budget-friendly upgrades and eco-friendly ideas. Could you     │
│                     │          │               │ share a specific example of a tip or project from the          │
│                     │          │               │ newsletter that you've found particularly helpful or           │
│                     │          │               │ inspiring?                                                     │
├─────────────────────┼──────────┼───────────────┼────────────────────────────────────────────────────────────────┤
│ 0                   │ 3        │ Respondent 1  │ Sure! One specific project that comes to mind is the article   │
│                     │          │               │ on upcycling old furniture. I had an old, worn-out coffee      │
│                     │          │               │ table that I was about to throw away, but your guide inspired  │
│                     │          │               │ me to give it a new life. The step-by-step instructions on     │
│                     │          │               │ sanding, painting, and adding new hardware were easy to        │
│                     │          │               │ follow, and now it looks like a brand-new piece! Plus, I love  │
│                     │          │               │ that it was an eco-friendly option. It saved me money and kept │
│                     │          │               │ the table out of the landfill.                                 │
├─────────────────────┼──────────┼───────────────┼──────

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook:

In [17]:
from edsl import Notebook

In [18]:
n = Notebook(path = "agentifying_responses.ipynb")

In [19]:
n.push(description = "Example code for creating agents for survey respondents and simulating follow-on interviews", visibility = "public")

{'description': 'Example code for creating agents for survey respondents and simulating follow-on interviews',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/c3b063a2-c815-4c7e-b026-058b21017fd1',
 'uuid': 'c3b063a2-c815-4c7e-b026-058b21017fd1',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [20]:
n = Notebook(path = "agentifying_responses.ipynb")

In [21]:
n.patch(uuid = "c3b063a2-c815-4c7e-b026-058b21017fd1", value = n)

{'status': 'success'}